# Étude à l'échelle de la consommation par adresse

Nous tâchons désormais de savoir s'il est possible pour un producteur d'électricité de prévoir la consommation d'électricité *d'une résidence* à partir d'un autre type de données plus local que des données départementales : des données à l'échelle des communes et des adresses. Nous continuons à considérer la météo comme une variable potentiellement pertinente pour comprendre la consommation, et nous cherchons à ajouter à cela la performance énergétique des bâtiments en passant par l'indice de DPE ([Diagnostic de performance énergétique](https://fr.wikipedia.org/wiki/Diagnostic_de_performance_%C3%A9nerg%C3%A9tique)). Ainsi, on cherche à savoir si les producteurs d'électricité, connaissant la notation énergétique d'un bâtiment toutes énergies comprises peuvent en prédire efficacement la consommation électrique de cette adresse au cours de l'année à venir.

Ce notebook s'organise en deux temps. 

- **Premier temps** : Récupération des données et formation d'un data frame regroupant toutes les données intéressantes.   
  Ce premier temps s'organise de la manière suivante :

      1. Récupération des données de consommation électriques individuelles sur le site d'Enedis
      2. Récupération des données météorologiques sur le site d'Infoclimat à l'échelle de communes, at agrégation de ces données pour constituer des moyennes annuelles
      3. Introduction d'une base de données de l'INSEE pour normaliser les noms de commune de la base de données météorologiques
      4. Constitution d'une liste de communes d'étude
      5. Récupération des données DPE
      6. Fusion des différentes données   

    
- **Second temps** : Régression utiles et conclusions

## Introduction - Mise en route

Le plan est le suivant : on cherche à régresser la consommation à l'échelle des adresses en fonction de la météo et de l'indice DPE de l'habitation. Nous cherchons donc une base de données d'adresses avec ces trois caractéristiques. Pour cela, nous allons checher à fusionner trois différentes bases de données :
- Des bases de données de consommations annuelles par adresse sur le site d'[Enedis](https://enedis.opendatasoft.com/explore/dataset/consommation-annuelle-residentielle-par-adresse/information/)
- Des bases de données météorologiques à l'échelle des communes sur le site d'[Infoclimat](https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois)
- Des bases de données de notation DPE par adresse fournie par l'[ADEME (Agence de l'environnement et de la maîtrise de l'énergie)](https://data.ademe.fr/datasets/dpe-france)

Pour permettre la bonne fusion de ces tables, nous importons en plus une base de données de l'INSEE via le module `cartiflette`, pour permettre d'associer à chaque commune son nom officiel et son numéro INSEE de commune.

Les bâtiments sur lesquels portera notre étude seront donc les résidences localisées dans des communes proposant des données météorologiques, et ayant renseignés à la fois leur indice DPE et leur consommation électrique annuelle.

On commence par installer puis importer toutes les librairies et tous les modules qui nous seront utiles.

In [17]:
!pip install pandas
!pip install geopandas
!pip install geoviews
!pip install lxml
!pip install urllib
!pip install matplotlib
!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a
!pip install mapclassify
!pip install folium

ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib
  Cloning https://github.com/inseefrlab/cartiflette (to revision 80b8a5a28371feb6df31d55bcc2617948a5f9b1a) to /tmp/pip-req-build-ocgnuvvs
  Running command git clone --filter=blob:none --quiet https://github.com/inseefrlab/cartiflette /tmp/pip-req-build-ocgnuvvs
  Running command git rev-parse -q --verify 'sha^80b8a5a28371feb6df31d55bcc2617948a5f9b1a'
  Running command git fetch -q https://github.com/inseefrlab/cartiflette 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Running command git checkout -q 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Resolved https://github.com/inseefrlab/cartiflette to commit 80b8a5a28371feb6df31d55bcc2617948a5f9b1a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [18]:
!pip install -q lxml

In [19]:
from declarations import *

In [20]:
import bs4
import lxml
import pandas as pd
import urllib
from tqdm.auto import tqdm

from urllib import request

## 1. Construction d'un tableau de données utile à notre étude 

### 1.1. Récupération des données de consommation électrique individuelle

Pour une étude sur la consommation électrique des bâtiments résidentiels, il paraît raisonnable de commencer par importer des donnés de consommation électrique. À cette fin, on utilise la fonction get_data_consumption, codée dans la page declarations.py , pour récupérer les données de consommation par adresse d'Enedis à partir d'URL, et ce pour quatre années différentes.

In [21]:
import declarations as dec

df_cons_2018=dec.get_data_consumption(dec.consumption_data_url_2018, "2018")
df_cons_2019=dec.get_data_consumption(dec.consumption_data_url_2019, "2019")
df_cons_2020=dec.get_data_consumption(dec.consumption_data_url_2020, "2020")
df_cons_2021=dec.get_data_consumption(dec.consumption_data_url_2021, "2021")

Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.
Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


/home/onyxia/work/Projet_Python/declarations.py:321: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


/home/onyxia/work/Projet_Python/declarations.py:321: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(path_to_data, sep=";")


Chargement des données, cette étape peut prendre quelques minutes
Téléchargement réussi.


On affiche ensuite l'une des tables pour montrer ce à quoi ressemblent nos données. On prend l'exemple de l'année 2021.

In [22]:
df_cons_2021

,Année,Code IRIS,Nom IRIS,Numéro de voie,Indice de répétition,Type de voie,Libellé de voie,Code INSEE de la commune,Nom de la commune,Segment de client,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Tri des adresses,Code EPCI,Code Département,Code Région
0,2021,800010101,Centre Ville Émonville,43.0,NaN,RUE,BOUCHER DE PERTHES,80001,Abbeville,RESIDENTIEL,11,59.439,5.404,3.546,43 RUE BOUCHER DE PERTHES,4,200070993.0,80.0,32.0
1,2021,800010502,Delique-Saint-Gilles,2.0,NaN,RUE,D ARTOIS,80001,Abbeville,RESIDENTIEL,31,115.704,3.732,3.546,2 RUE D ARTOIS,9,200070993.0,80.0,32.0
2,2021,800010401,Faubourg de la Bouvaque,30.0,NaN,RUE,DE LA CITE LEDAY,80001,Abbeville,RESIDENTIEL,12,36.454,3.038,3.546,30 RUE DE LA CITE LEDAY,17,200070993.0,80.0,32.0
3,2021,800010502,Delique-Saint-Gilles,8.0,NaN,RUE,DE NORMANDIE,80001,Abbeville,RESIDENTIEL,10,25.769,2.577,3.546,8 RUE DE NORMANDIE,23,200070993.0,80.0,32.0
4,2021,800010502,Delique-Saint-Gilles,20.0,NaN,RUE,DE NORMANDIE,80001,Abbeville,RESIDENTIEL,10,22.995,2.299,3.546,20 RUE DE NORMANDIE,26,200070993.0,80.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407048,2021,751072705,École Militaire 5,15.0,NaN,AVENUE,DE BRETEUIL,75056,Paris,RESIDENTIEL,16,55.216,3.451,3.013,15 AVENUE DE BRETEUIL,250491,200054781.0,75.0,11.0
407049,2021,751072705,École Militaire 5,14.0,NaN,AVENUE,DE BRETEUIL,75056,Paris,RESIDENTIEL,17,61.842,3.638,3.013,14 AVENUE DE BRETEUIL,250492,200054781.0,75.0,11.0
407050,2021,682240903,Coteaux Sud-Ouest,8.0,NaN,RUE,JULES VERNE,68224,Mulhouse,RESIDENTIEL,15,23.942,1.596,2.944,8 RUE JULES VERNE,210007,200066009.0,68.0,44.0
407051,2021,682240601,Nordfeld Nord,84.0,NaN,RUE,LAURENT,68224,Mulhouse,RESIDENTIEL,12,32.852,2.738,2.944,84 RUE LAURENT,210022,200066009.0,68.0,44.0


### 1.2. Scrapping des données météo communales

Une fois les données de consommation électrique récupérées, nous souhaitons nous concentrer sur des données météorologiques à l'échelle la plus locale qu'il nous est possible d'obtenir. Le site Info Climat propose justement des données météorologiques relevées dans plusieurs stations françaises, à une fréquence quotidienne. Par simplicté, nous supposerons que les rélevés météorologiques d'une station portent sur toute la commune dans laquelle est placée la station. 

Les données que nous récupérons sont des moyennes mensuelles des relevés quotidiens. Cependant, nous n'avons accès qu'à des données de consommations individuelles *annuelles*. Pour donner du sens à ces données météorologiques dans notre étude, nous effectuons donc des moyennes annuelles à partir de ces données mensuelles. 

En régressant la consommation par des données météo d'un pas de temps aussi large, on se demande si le "contexte" climatique d'une région ou d'une époque influe sur la consommation d'électricité des ménages. Ce lien pourrait avoir du sens car intuitivement un climat plus froid demande plus de chauffage, et un environnement plus sombre demande plus d'écairage. 

Pour ordonner les différentes fonctions qui se rapportent à l'extraction de données météorologiques à l'échelle des communes, on se propose de construire une classe regroupant les différentes fonctions qui nous intéressent.

In [131]:
class Meteo :
    """
    Class permettant de scrapper et d'effectuer des opérations sur les données d'Infoclimat concernant la température observée par différentes 
    stations météo.
    """
    def __init__(self):
        """
        Initialisation de la class contenant :
            self.année_début : int
                Année de début des données qu'on souhaite scrapper
            self.année_fin : int
                Année de fin des données qu'on souhaite scrapper
            self.data_all : dict
                Contient l'ensemble des données par année (initialement vides)
        """
        #Par défaut, on cadre notre extraction de données entre 2018 et 2021, années pour lesquelles nous avons des données de consommation
        self.année_début=2018 
        self.année_fin=2021
        self.data_all = {i : {j : {} for j in range(1,13)} for i in range(self.année_début,self.année_fin+1)}

    def scrap(self):
        """
        Fonction permettant de scrapper les données voulues, i.e. les données d'infoclimat concernant la température observée par différentes 
        stations météo entre 2011 et 2018, avec un pré-tri permettant d'éviter toute répétition de ville au sein d'un même mois.

            Paramètres:
            ------------
                self.data_all : dict
                    Dictionnaire contenant toutes les données par années
        
            Output:
            ----------
                self.data_pop : dict
                    Dictionnaire contenant toutes les données par années actualisées
            
        """
        #On boucle sur toutes les années souhaitées
        for y in self.data_all :
            #On boucle ensuite sur tous les mois de l'année
            for m in self.data_all[y] :
                url_temp = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
                raw_text = request.urlopen(url_temp).read()
                page = bs4.BeautifulSoup(raw_text,'lxml')
                tableau = page.find('table', {'id' : 'tableau-releves'})
                rows = tableau.find_all('tr')
                
                #On stocke les données d'un mois et d'une année dans un dictionnaire
                df = {
            'ville' : [rows[i].find('a').text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text],
            'tnn' : [rows[i].find('div').text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text],
            'tnm' : [rows[i].find_all('td')[2].text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text],
            'tmm' : [rows[i].find_all('td')[3].text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text],
            'txm' : [rows[i].find_all('td')[4].text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text],
            'txx' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text],
            'rr' : [rows[i].find_all('td')[6].text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text],
            "ens" : [rows[i].find_all('td')[7].text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text],
            'rafale' : [rows[i].find_all('td')[8].text for i in range(1,len(rows)) if rows[i].find('a').text!=rows[i-1].find('a').text] 
                    }
                #La condition "if" dans toutes les listes permet de nous assurer que nous ne prenons qu'une seule donnée par ville
                #Par défaut, si cette ville est renseignée deux fois, nous ne prenons que la première information
                #Ceci nous permet d'éviter les répétitions de villes, qui bruitent la suite
               
                self.data_all[y][m] = df

    def export(self):
        """
        Fonction intégrée d'export au format DataFrame et au format .json
        pour permettant de faciliter les manipulations.
        Cette méthode ne sera pas utilisée en pratique dans notre projet

        Paramètres:
        ------------
            self.data_pop : dict
                Dictionnaire contenant toutes les données par années
        
        Output:
        ----------
            Aucun
        """
        
        self.df_all = pd.DataFrame(self.data_all)
        self.df_all.to_json('data_base.json')

    def liste_min(self, year):
        """
        Comme les stations ne sont pas les mêmes d'un mois à l'autre,
        cette fonction permet, pour une année donnée, de construire la liste des stations communes 
        à tous les mois pour une année donnée.

        Entrée:
        ------------
            year : int
                Année qui nous intéresse. Elle doit être entre self.année_début=2018 et self.année_fin

        Paramètres:
        ------------
            self.data_all : dict
                Dictionnaire contenant toutes les données par années
        
        Output:
        ----------
            intersection : list
                liste des stations communes à tous les mois pour une année donnée.
        """
        intersection=self.data_all[year][1]['ville']
        for m in range(2,13):
            intersection=[x for x in intersection if x in self.data_all[year][m]['ville']]
        return intersection

    def dico_minimal(self,year) :
        """
        Cette fonction construit le dictionnaire des données pour chaque mois d'une année "year" donnée, 
        mais seulement pour les villes proposant des données pour tous les mois de l'année.

        Entrée:
        ------------
            year : int
                Année qui nous intéresse. Elle doit être entre self.année_début=2018 et self.année_fin

        Paramètres:
        ------------
            self.data_all : dict
                Dictionnaire contenant toutes les données par années
            self.list_min : function
                Méthode de la classe Meteo qui renvoie la liste des villes communes à tous les mois 
                pour une année donnée
        
        Output:
        ----------
            df_min : dict
                dictionnaire des données pour chaque mois d'une année "year" donnée, 
                mais seulement pour les villes pour lesquelles on a des données pour tous les mois
        """
        intersection = self.liste_min(year)
        df_min={j : {} for j in range(1,13)}
        for m in range (1,13):
            df={'ville':[],'tnn':[],'tnm':[],'tmm':[],'txm':[],'txx':[],'rr':[],"ens":[],'rafale':[]}
            for i in range (len(self.data_all[year][m]['ville'])):
                if self.data_all[year][m]['ville'][i] in intersection and self.data_all[year][m]['ville'][i] != self.data_all[year][m]['ville'][i-1]: 
                    df['ville'].append(self.data_all[year][m]['ville'][i])
                    df['tnn'].append(self.data_all[year][m]['tnn'][i])
                    df['tnm'].append(self.data_all[year][m]['tnm'][i])
                    df['tmm'].append(self.data_all[year][m]['tmm'][i])
                    df['txm'].append(self.data_all[year][m]['txm'][i])
                    df['txx'].append(self.data_all[year][m]['txx'][i])
                    df['rr'].append(self.data_all[year][m]['rr'][i])
                    df['ens'].append(self.data_all[year][m]['ens'][i])
                    df['rafale'].append(self.data_all[year][m]['rafale'][i])
            df_min[m]=df       
        return df_min
        
        

    def tableau_annuel(self,year):
        """
        Fonction intégrée d'export en data frame des données météo sous forme de moyenne pour une année.
        Il faut que cette année "year" ait été importée, c'est-à-dire qu'elle soit 
        entre self.année_début et self.année_fin

        Paramètres:
        ------------
            self.dico_minimal(year) : dict
                Dictionnaire contenant toutes les données par mois pour les stations aux données fournies
                pour toute l'année
            self.dico_minimal(year) : list
                Liste des stations offrant des données pour toute l'année
        
        Output:
        ----------
            data : DataFrame
        """
        dico_min=self.dico_minimal(year)
        L=self.liste_min(year)
        df={j : {} for j in ['ville','tnn','tnm','tmm','txm','txx','rr','ens','rafale']}
        for x in range(len(L)):
            df['ville'][x]=L[x]
        for j in ['tnn','tnm','tmm','txm','txx','rr','ens','rafale']:
            dico={ville : [] for ville in L} 
            for x in range(len(L)):
                for m in range(1,13):
                    u=0
                    if dico_min[m][j][x]!='':
                        u+=1
                        dico[L[x]].append(float(dico_min[m][j][x]))
                    if u==0 :
                        moyenne='Rien'
                    else :
                        moyenne = sum(dico[L[x]])/u
                df[j][x]=moyenne
        data= pd.DataFrame(df)
        data.rename({'ville' : 'Villes', 
            'tnn' : 'Température minimale extrême du mois',
            'tnm' : 'Moyenne des températures minimales du mois', 
            'tmm' : 'Température moyenne du mois', 
            'txm' : 'Moyenne des températures maximales du mois', 
            'txx' : 'Température maximale extrême du mois', 
            'rr' : 'Cumul de précipitation du mois', 
            'ens' : "Heure d'ensoleillement du mois", 
            'rafale' : 'Rafale maximale du mois'},axis=1,inplace = True)
        return(data)
        

    def rename_pratique(self):
        """
        Fonction intégrée permettant de renommer facilement les colonnes par le nom utilisé par Infoclimat dans le but de faciliter le traitement.

        Paramètres :
        ------------
            self.df_all : DataFrame
                DataFrame contenant toutes les données météo extraites

        Output :
        --------
            self.df_all : DataFrame
                DataFrame contenant les mêmes données mais dont les colonnes ont été renommées
        """
        
        self.df_all = pd.DataFrame(self.data_all)
        self.df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensoleillement du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)

    def rename_esthétique(self):
        """
        Fonction intégrée permettant de renommer facilement les colonnes par leur véritable nom, après les avoir simplifié.

        Paramètres :
        ------------
            self.df_all : DataFrame
                DataFrame contenant toutes les données météo extraites

        Output :
        --------
            self.df_all : DataFrame
                DataFrame contenant les mêmes données mais dont les colonnes ont été renommées
        """
        
        self.df_all = pd.DataFrame(self.data_all)
        self.df_all.rename({'ville' : 'Villes', 
            'tnn' : 'Température minimale extrême du mois',
            'tnm' : 'Moyenne des températures minimales du mois', 
            'tmm' : 'Température moyenne du mois', 
            'txm' : 'Moyenne des températures maximales du mois', 
            'txx' : 'Température maximale extrême du mois', 
            'rr' : 'Cumul de précipitation du mois', 
            'ens' : "Heure d'ensoleillement du mois", 
            'rafale' : 'Rafale maximale du mois'},axis=1,inplace = True)

    

In [23]:
import declarations
#Importation de la class Meteo
lameteo=declarations.Meteo()
#On récupère les données
lameteo.scrap()
#On exporte au format DataFrame
lameteo.export()

**Remarque** : On présente ici une rapide parenthèse pour montrer à quel point il est important d'éviter les répétitions de villes et de s'assurer que la station renseigne bien les données météo pour tous les mois de l'année. En effet, dans la cellule ci-dessous, on voit que lorsque qu'on demande le nombre de villes (donc le nombre de stations) avec des informations pour chaque mois, et bien il varie. C'est le cas pour 2019, mais aussi pour toutes les autres années d'études.

In [24]:
[len(lameteo.df_all[2019][i]['ville']) for i in range(1,13)] #Exemple de l'année 2019

[888, 884, 959, 961, 969, 969, 970, 973, 978, 981, 988, 991]

In [25]:
[len(lameteo.df_all[2020][i]['ville']) for i in range(1,13)] #Exemple de l'année 2020

[980, 1432, 1329, 1292, 1320, 1327, 1323, 1329, 1341, 1338, 1346, 1359]

Maintenant que la classe est créée, nous pouvons exporter les données pour voir ce à quoi elles ressemblent. Comme pour les données de consommation, on prend l'exemple de l'année 2021. 

In [27]:
meteo_2021=lameteo.tableau_annuel(2021)
meteo_2021

,Villes,Température minimale extrême du mois,Moyenne des températures minimales du mois,Température moyenne du mois,Moyenne des températures maximales du mois,Température maximale extrême du mois,Cumul de précipitation du mois,Heure d'ensoleillement du mois,Rafale maximale du mois
0,Abbeville (80),23.6,88.7,133.5,177.5,264.8,17.4,Rien,Rien
1,Abbeville - centre (80),14.9,87.6,136.1,184.5,270.0,695.8,Rien,460.2
2,ACHERES (78),-4.3,79.3,140.0,200.3,296.2,696.0,Rien,Rien
3,ADAST (65),14.7,82.8,151.4,219.3,325.0,841.5,Rien,Rien
4,Agde (34),79.6,143.3,192.1,240.8,314.3,325.2,Rien,633.9
...,...,...,...,...,...,...,...,...,...
1286,[MAE] Lycée Emile Zola - AIX EN PROVENCE (13),29.9,70.6,131.8,192.9,321.6,576.4,1850.5,477.9
1287,[MAE] Lycée Marguerite Filhol - FUMEL (47),27.0,102.3,160.8,219.4,311.5,891.2,1766.6,576.0
1288,[MAE] Lycée Parc de Vilgénis - MASSY (91),22.9,93.5,147.3,202.1,305.5,Rien,1016.1,506.8
1289,[MAE] Lycée Saint Exupéry - LA ROCHELLE (17),55.2,Rien,Rien,Rien,163.2,447.2,966.9,492.3


### 1.3. Utilisation d'une base intermédiaire pour normaliser les noms de communes

Dans le tableau des données météo précédent, on remarque que certains noms de stations correspondent bien à des noms de ville, mais ça n'est pas le cas de tous. Pour simplifier le croisement avec les autres tableaux de données, nous allons donc importer un tableau de l'INSEE, permettant d'associer quand c'est possible le nom d'une station avec le code INSEE de la commune ainsi que ses données géographiques.

In [28]:
communes = s3.download_vectorfile_url_all(
    values = "metropole",
    crs = 4326,
    borders = "COMMUNE",
    vectorfile_format="topojson",
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

communes["area"] = dep.to_crs(2154).area

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 70.4MiB [00:01, 56.1MiB/s]


In [29]:
communes

,id,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,source,territoire,geometry,area
0,COMMUNE_0000000009754033,NaN,Connangles,CONNANGLES,43076,Commune simple,137,11,1,43,84,200073419,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((3.61421 45.27206, 3.61377 45.27255, ...",5.774291e+09
1,COMMUNE_0000000009760784,NaN,Vidouze,VIDOUZE,65462,Commune simple,243,13,3,65,76,200072106,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-0.02485 43.43002, -0.02500 43.43012...",7.418552e+09
2,COMMUNE_0000000009742077,NaN,Fouesnant,FOUESNANT,29058,Commune simple,9864,11,4,29,53,242900660,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((-3.97912 47.70392, -3.97897 47...",4.033539e+09
3,COMMUNE_0000000009735245,NaN,Plougrescant,PLOUGRESCANT,22218,Commune simple,1166,27,3,22,53,200065928,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((-3.20273 48.84902, -3.20259 48...",4.720673e+09
4,COMMUNE_0000000009752504,NaN,Montcarra,MONTCARRA,38250,Commune simple,569,24,2,38,84,200068542,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((5.43220 45.61477, 5.43220 45.61438, ...",7.365673e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34821,COMMUNE_0000000009742349,NaN,Brie,BRIE,35041,Commune simple,1006,12,1,35,53,243500634,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-1.58072 47.94312, -1.58057 47.94420...",NaN
34822,COMMUNE_0000000009742038,NaN,Orgères,ORGERES,35208,Commune simple,5152,12,3,35,53,243500139,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((-1.64908 47.96488, -1.65011 47.96595...",NaN
34823,COMMUNE_0000000009734653,NaN,Castillon-en-Auge,CASTILLON-EN-AUGE,14141,Commune simple,163,19,3,14,28,200069532,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((0.09085 49.04872, 0.09129 49.04872, ...",NaN
34824,COMMUNE_0000000009734657,NaN,Saint-Pierre-en-Auge,SAINT-PIERRE-EN-AUGE,14654,Commune simple,7329,18,3,14,28,200069532,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"POLYGON ((0.10231 49.02482, 0.10246 49.02443, ...",NaN


On donne la liste des colonnes ainsi que le nombres d'éléments distincts que chacune contient pour se faire une idée plus précise de la base.

In [30]:
communes.nunique()

id            34826
ID                0
NOM           32596
NOM_M         32537
INSEE_COM     34826
STATUT            5
POPULATION     5786
INSEE_CAN        56
INSEE_ARR         9
INSEE_DEP        96
INSEE_REG        13
SIREN_EPCI     1244
source            1
territoire        1
geometry      34826
area             96
dtype: int64

### 1.4. Création d'une liste de villes d'étude

On garde pour la suite le nom des stations utiles. Ces noms nous permettront de filtrer les communes dont nous devons étudier les DPE.

On voit que les noms de stations ne sont pas très propres : ils incluent les numéros de département, voire incluent des quartiers ou regroupent plusieurs villes. On cherche donc à nettoyer ces noms, pour qu'ils puissent être comparés aux noms de communes donnés par l'API utile pour les DPE. 

On ne touche pour cela qu'à la liste "Villes" du data frame. Nous remplacerons ensuite la base initiale par la base nettoyée, et supprimons les lignes qui ne nous intéressent pas dans le data frame après nettoyage. 

L'API des DPE qui nous intéresse n'identifie une commune que par son code INSEE. Il faut donc que, dans la base de données de chaque station, je réussisse à fusionner la base météo avec une base de données qui comprend les codes communes INSEE à partir de la commune de la station.

Par simplicité, nous ne garderons que les stations dont le nom est un nom de commune officiel ; nous ne chercherons pas pour chaque station la commune à laquelle elle se rapporte.

In [41]:
#On ne garde que les stations dans les villes et pas celles de stations de "Météo à l'école" (MAE)
def sans_mae(year):
    '''Cette fonction supprime les listes du data frame correspondant aux stations MAE
    Argument : year (type : int, année voulue) 
    Sortie : L[~L['Villes'].str[0].isin(['['])] (type : DataFrame)'''
    L=lameteo.tableau_annuel(year)
    return L[~L['Villes'].str[0].isin(['['])]
#meteo_bis_2021=meteo_2021[~meteo_2021['Villes'].str[0].isin(['['])]
meteo_bis_2021=sans_mae(2021)
meteo_bis_2021

,Villes,Température minimale extrême du mois,Moyenne des températures minimales du mois,Température moyenne du mois,Moyenne des températures maximales du mois,Température maximale extrême du mois,Cumul de précipitation du mois,Heure d'ensoleillement du mois,Rafale maximale du mois
0,Abbeville (80),23.6,88.7,133.5,177.5,264.8,17.4,Rien,Rien
1,Abbeville - centre (80),14.9,87.6,136.1,184.5,270.0,695.8,Rien,460.2
2,ACHERES (78),-4.3,79.3,140.0,200.3,296.2,696.0,Rien,Rien
3,ADAST (65),14.7,82.8,151.4,219.3,325.0,841.5,Rien,Rien
4,Agde (34),79.6,143.3,192.1,240.8,314.3,325.2,Rien,633.9
...,...,...,...,...,...,...,...,...,...
1255,Xonrupt-Longemer (88),-35.8,42.5,95.5,148.6,255.1,1428.2,Rien,382.8
1256,Xonrupt-Longemer - Le Poli (88),-27.1,47.3,95.5,143.5,249.1,1906.5,1212.1,346.0
1257,Xonrupt-Longemer - Refuge du Sotré (88),-38.2,37.8,68.7,99.2,207.2,Rien,Rien,942.9
1258,Yvoy-le-Marron (41),-3.5,76.9,137.5,197.7,298.0,85.6,Rien,Rien


On voit qu'en plus par ville il peut il y avoir plusieurs stations (correspondant à plusieurs quartiers). On va donc essayer de supprimer les parties qui ne dénotent pas de la ville dans les noms de stations.

In [32]:
def nettoyage(liste):
    '''Cette fonction est censée séparer les numéros de département des villes dans une liste présentée comme la liste liste_communes
    Argument : liste (type : list ; liste de string) 
    Sortie : List (type : list ; liste de listes)'''
    List=[]
    for ville in liste:
        i=0
        mot=''
        while i < len(ville) and ville[i+1] != '(' :
            mot+=ville[i]
            i+=1
        mot=mot.upper()
        List.append(mot)
    
    return List

In [33]:
meteo_bis_2021['Villes']=pd.Series(nettoyage(meteo_bis_2021['Villes'].to_list()))

/tmp/ipykernel_300/349770013.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meteo_bis_2021['Villes']=pd.Series(nettoyage(meteo_bis_2021['Villes'].to_list()))


In [34]:
meteo_bis_2021['Villes']

0                                ABBEVILLE
1                      ABBEVILLE - CENTRE 
2                                  ACHERES
3                                    ADAST
4                                     AGDE
                       ...                
1255                      XONRUPT-LONGEMER
1256            XONRUPT-LONGEMER - LE POLI
1257    XONRUPT-LONGEMER - REFUGE DU SOTRÉ
1258                        YVOY-LE-MARRON
1259                                YZERON
Name: Villes, Length: 1260, dtype: object

In [35]:
meteo_bis_2021

,Villes,Température minimale extrême du mois,Moyenne des températures minimales du mois,Température moyenne du mois,Moyenne des températures maximales du mois,Température maximale extrême du mois,Cumul de précipitation du mois,Heure d'ensoleillement du mois,Rafale maximale du mois
0,ABBEVILLE,23.6,88.7,133.5,177.5,264.8,17.4,Rien,Rien
1,ABBEVILLE - CENTRE,14.9,87.6,136.1,184.5,270.0,695.8,Rien,460.2
2,ACHERES,-4.3,79.3,140.0,200.3,296.2,696.0,Rien,Rien
3,ADAST,14.7,82.8,151.4,219.3,325.0,841.5,Rien,Rien
4,AGDE,79.6,143.3,192.1,240.8,314.3,325.2,Rien,633.9
...,...,...,...,...,...,...,...,...,...
1255,XONRUPT-LONGEMER,-35.8,42.5,95.5,148.6,255.1,1428.2,Rien,382.8
1256,XONRUPT-LONGEMER - LE POLI,-27.1,47.3,95.5,143.5,249.1,1906.5,1212.1,346.0
1257,XONRUPT-LONGEMER - REFUGE DU SOTRÉ,-38.2,37.8,68.7,99.2,207.2,Rien,Rien,942.9
1258,YVOY-LE-MARRON,-3.5,76.9,137.5,197.7,298.0,85.6,Rien,Rien


On cherche à savoir combien de villes on est amenés à garder si l'on ne veut pas trop se casser la tête et ne garder que les noms "jolis" de stations, pour lesquels on peut facilement trouver le nom de la ville d'origine.

In [36]:
filtre=meteo_bis_2021.merge(communes, left_on='Villes', right_on='NOM_M')[['NOM_M','STATUT','POPULATION','INSEE_COM','geometry']]
filtre

,NOM_M,STATUT,POPULATION,INSEE_COM,geometry
0,ABBEVILLE,Sous-préfecture,22980,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ..."
1,ACHERES,Commune simple,21529,78005,"POLYGON ((2.04953 48.95702, 2.04938 48.95770, ..."
2,ACHERES,Commune simple,380,18001,"POLYGON ((2.44015 47.27926, 2.44015 47.27965, ..."
3,ADAST,Commune simple,297,65001,"POLYGON ((-0.06867 42.97044, -0.07087 42.97014..."
4,AGDE,Commune simple,29600,34003,"MULTIPOLYGON (((3.50101 43.26349, 3.50130 43.2..."
...,...,...,...,...,...
751,WINTERSBOURG,Commune simple,269,57747,"POLYGON ((7.19741 48.77898, 7.19668 48.77917, ..."
752,XERTIGNY,Commune simple,2586,88530,"POLYGON ((6.45837 48.04351, 6.45807 48.04126, ..."
753,XONRUPT-LONGEMER,Commune simple,1515,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ..."
754,YVOY-LE-MARRON,Commune simple,760,41297,"POLYGON ((1.84782 47.58568, 1.84724 47.58529, ..."


Après ce filtre, il nous reste donc 755 communes d'études, ce qui reste satisfaisant : on est rassuré que notre tri très simple n'ait pas enlevé trop de villes d'étude.

Maintenant, nous fusionnons cette base triée de données météo avec la base des données de consommation.

In [37]:
L=df_cons_2021['Nom de la commune'].to_list()
df_cons_2021['Nom de la commune']=pd.Series([x.upper() for x in L])
filtre_conso_2021 = filtre.merge(df_cons_2021, left_on="NOM_M", right_on="Nom de la commune")[['NOM_M','STATUT','INSEE_COM','geometry','Code IRIS', 'Nom IRIS', 'Nombre de logements', "Consommation annuelle totale de l'adresse (MWh)", "Consommation annuelle moyenne par logement de l'adresse (MWh)", "Consommation annuelle moyenne de la commune (MWh)", "Adresse"]]   
filtre_conso_2021

,NOM_M,STATUT,INSEE_COM,geometry,Code IRIS,Nom IRIS,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse
0,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010101,Centre Ville Émonville,11,59.439,5.404,3.546,43 RUE BOUCHER DE PERTHES
1,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,31,115.704,3.732,3.546,2 RUE D ARTOIS
2,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010401,Faubourg de la Bouvaque,12,36.454,3.038,3.546,30 RUE DE LA CITE LEDAY
3,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,10,25.769,2.577,3.546,8 RUE DE NORMANDIE
4,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint-Gilles,10,22.995,2.299,3.546,20 RUE DE NORMANDIE
...,...,...,...,...,...,...,...,...,...,...,...
35160,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),52,110.382,2.123,5.875,224 CHEMIN DE L HERMINE
35161,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),10,20.775,2.078,5.875,478 ROUTE DES PERGIS
35162,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),14,85.812,6.129,5.875,209 ROUTE DES PERGIS
35163,XONRUPT-LONGEMER,Commune simple,88531,"POLYGON ((7.00967 48.04292, 7.00791 48.04263, ...",885310000,Xonrupt-Longemer (commune non irisée),12,37.800,3.150,5.875,65 IMPASSE DU GRAND RAIN


In [55]:
len(filtre_conso_2021['NOM_M'].unique())

249

In [69]:
liste_communes=filtre_conso_2021['INSEE_COM'].unique()
len(liste_communes)

322

Attention ! On remarque ici que notre base de données comporte beaucoup de villes homonymes. Il faudrait donc bien faire attention au code INSEE / au code postal d'une ville pour s'assurer qu'on ne la confond pas avec une autre ville du même nom.

Notre étude portera donc au plus sur des immeubles issus de 249 communes différentes !

On essaiera par la suite de trouver un moyen de relier l'adresse de la base de données des DPE à l'adresse de la base qu'on vient dd définir. Pour cela, on cherchera à normaliser l'adresse de notre base.

In [78]:
filtre_conso_2021['Adresse normalisée']=filtre_conso_2021['Adresse']+ ' '+ filtre_conso_2021['INSEE_COM']+ ' '+ filtre_conso_2021['NOM_M']
filtre_conso_2021

,NOM_M,STATUT,INSEE_COM,geometry,Code IRIS,Nom IRIS,Nombre de logements,Consommation annuelle totale de l'adresse (MWh),Consommation annuelle moyenne par logement de l'adresse (MWh),Consommation annuelle moyenne de la commune (MWh),Adresse,Adresse normalisée
0,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010501,La Porte au Bois,28,134.006,4.786,3.522,39 ROUTE D AMIENS,39 ROUTE D AMIENS 80001 ABBEVILLE
1,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,36,91.020,2.528,3.522,10 RUE DE GASCOGNE,10 RUE DE GASCOGNE 80001 ABBEVILLE
2,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010503,Ponthieu-Bagatelle,10,18.327,1.833,3.522,5 RUE DE GUYANE,5 RUE DE GUYANE 80001 ABBEVILLE
3,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,10,37.592,3.759,3.522,15 RUE DE NORMANDIE,15 RUE DE NORMANDIE 80001 ABBEVILLE
4,ABBEVILLE,Sous-préfecture,80001,"POLYGON ((1.83974 50.07802, 1.83856 50.07744, ...",800010502,Delique-Saint Gilles,10,33.586,3.359,3.522,13 RUE DE PICARDIE,13 RUE DE PICARDIE 80001 ABBEVILLE
...,...,...,...,...,...,...,...,...,...,...,...,...
25167,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,10,12.318,1.232,3.968,10 AVENUE DU MARECHAL DE VAUX,10 AVENUE DU MARECHAL DE VAUX 43268 YSSINGEAUX
25168,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,12,10.403,0.867,3.968,165 RUE LOUIS JOUVET,165 RUE LOUIS JOUVET 43268 YSSINGEAUX
25169,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,12,11.531,0.961,3.968,85 RUE LOUIS JOUVET,85 RUE LOUIS JOUVET 43268 YSSINGEAUX
25170,YSSINGEAUX,Sous-préfecture,43268,"POLYGON ((4.09598 45.09519, 4.09598 45.09529, ...",432680102,Sud,18,17.056,0.948,3.968,200 RUE SAINT ROCH,200 RUE SAINT ROCH 43268 YSSINGEAUX


### 1.5. Récupération des données de Diagnostics de Performance Energétique (DPE)

On commence par programmer une fonction qui permet de récupérer des données à partir d'une APL. Voici l'exemple de Lino lorsqu'on essaye la demande avec une seule ville.

In [7]:
code_commune = "01450"
size = 100
api_root = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines"
url_api = f"{api_root}?page=1&after=10&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=neighbors"

In [70]:
import requests
import pandas as pd
import geopandas as gpd

def get_dpe_from_url(api_url):
    '''Argument:
        api_url (string) : url de l'API voulue
    Sortie:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe

dpe = get_dpe_from_url(url_api)
dpe

,classe_consommation_energie,tr001_modele_dpe_type_libelle,annee_construction,_geopoint,latitude,surface_thermique_lot,_i,tr002_type_batiment_description,geo_adresse,_rand,...,classe_estimation_ges,nom_methode_dpe,tv016_departement_code,consommation_energie,date_etablissement_dpe,longitude,_score,_id,version_methode_dpe,geometry
0,D,Vente,1947,"45.925922,5.229964",45.925922,117.16,487,Maison Individuelle,Rue de la Brugnière 01800 Villieu-Loyes-Mollon,23215,...,E,Méthode Facture,01,178.00,2013-06-13,5.229964,None,04JZNel3WCJYcfsHpCcHv,NaN,POINT (5.22996 45.92592)
2,D,Neuf,2006,"45.923421,5.223777",45.923421,90.53,689,Maison Individuelle,Chemin du Pont-vieux 01800 Villieu-Loyes-Mollon,401672,...,C,FACTURE - DPE,01,227.99,2013-06-11,5.223777,None,rkdV2lJn2wxaidVBaHBFY,V2012,POINT (5.22378 45.92342)
3,D,Neuf,2006,"45.923068,5.223271",45.923068,105.27,690,Maison Individuelle,Lotissement le Pont Vieux 01800 Villieu-Loyes-...,428929,...,C,FACTURE - DPE,01,216.38,2013-06-11,5.223271,None,P92FDkFFgWqoktiPEptkC,V2012,POINT (5.22327 45.92307)
6,E,Vente,1983,"45.91956,5.224214",45.919560,130.00,1127,Maison Individuelle,Rue du Pollet 01800 Villieu-Loyes-Mollon,848537,...,C,Méthode 3CL,01,309.00,2013-07-09,5.224214,None,jBNxOg77oHrmTW6EIyTR5,"3CL-DPE, version 1.3",POINT (5.22421 45.91956)
7,E,Vente,2006,"45.916251,5.221846",45.916251,90.34,1374,Maison Individuelle,6 Rue des Terres du Pollet 01800 Villieu-Loyes...,598270,...,C,3CL - DPE,01,284.00,2013-07-16,5.221846,None,MuAKfgQyjAM0GHTLgErrm,2012,POINT (5.22185 45.91625)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,C,Location,2019,"45.920228,5.222557",45.920228,54.82,102717,Logement,25 Rue du Bottet 01800 Villieu-Loyes-Mollon,235935,...,D,3CL - DPE,01,94.36,2020-10-21,5.222557,None,bOsqqc-zpzRKA7u_8Pdj7,V2012,POINT (5.22256 45.92023)
96,B,Location,2019,"45.920228,5.222557",45.920228,65.89,102718,Logement,25 Rue du Bottet 01800 Villieu-Loyes-Mollon,140973,...,C,3CL - DPE,01,74.80,2020-10-21,5.222557,None,rgH9CYqZTZuJCxurcUaub,V2012,POINT (5.22256 45.92023)
97,B,Location,2019,"45.920292,5.223036",45.920292,67.77,102719,Logement,257 Avenue Charles de Gaulle 01800 Villieu-Loy...,912454,...,C,3CL - DPE,01,65.00,2020-10-21,5.223036,None,l1yvcYwWAV9UQtnTg48nb,V2012,POINT (5.22304 45.92029)
98,C,Location,2019,"45.920292,5.223036",45.920292,58.23,102720,Logement,257 Avenue Charles de Gaulle 01800 Villieu-Loy...,755526,...,D,3CL - DPE,01,102.29,2020-10-21,5.223036,None,k4E_1qA9XKd6Dps8ceBZf,V2012,POINT (5.22304 45.92029)


In [69]:
list(dpe)

['classe_consommation_energie',
 'tr001_modele_dpe_type_libelle',
 'annee_construction',
 '_geopoint',
 'latitude',
 'surface_thermique_lot',
 '_i',
 'tr002_type_batiment_description',
 'geo_adresse',
 '_rand',
 'code_insee_commune_actualise',
 'estimation_ges',
 'geo_score',
 'classe_estimation_ges',
 'nom_methode_dpe',
 'tv016_departement_code',
 'consommation_energie',
 'date_etablissement_dpe',
 'longitude',
 '_score',
 '_id',
 'version_methode_dpe',
 'geometry']

Il est trop coûteux de récupérer l'intégralité des données de DPE, et nous ne disposons de données météo que pour certaines communes. L'idée est donc de ne récupérer les données de DPE que pour les communes voulues et de les agréger dans une unique table de données.

In [72]:
liste_code_commune = filtre['INSEE_COM']
def gros_data_frame(liste_commune):
    DPE=pd.DataFrame(columns=[list(dpe)])
    for i in liste_commune: 
        code_commune=i
        size = 100
        api_root = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines"
        url_api = f"{api_root}?page=1&after=10&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + f"&size={size}&select=" + "%2A&sampling=neighbors"
        DPE=pd.concat([DPE,get_dpe_from_url(url_api)])


big_data=gros_data_frame(liste_code_commune)
big_data

AttributeError: 'DataFrame' object has no attribute 'longitude'

In [200]:
import time
def api_ademe(codes_insee, nbr_page = 10):
    
    base_url = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines?page="
    
    for code_insee in codes_insee :
        for i in range(1,10*nbr_page + 2,10):
            time.sleep(5)
            page_init = i
            page_suiv = i + 9
    
            url = base_url + f"{page_init}" + "&after=" + f"{page_suiv}" + "&format=json&qs=code_insee_commune_actualise:" + code_insee
            req_api = requests.get(url)
            df = pd.json_normalize(req_api.json()['results'])
            data.append(df[~df['geo_adresse'].isna()])

    return pd.concat(data,axis=0,ignore_index=True)

filtre_conso_2021['Villes']

In [64]:
def get_json_results(data, code_commune):
    updated_data = data.copy()
    api_root = "https://koumoul.com/data-fair/api/v1/datasets/dpe-france/lines"
    url_api = f"{api_root}?page=1&after=10&format=json&q_mode=simple&qs=code_insee_commune_actualise" + "%3A%22" + f"{code_commune}" + "%22" + "&select=" + "%2A&sampling=neighbors"
    req=requests.get(url_api)
    wb=req.json()
    updated_data += wb['results']
    return updated_data

def api_ademe(codes_insee):
    data=[]
    for ville in codes_insee :
        data=get_json_results(data, ville)
    #return data
    df = pd.json_normalize(data)
    #return data
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe



In [65]:
api=api_ademe(liste_communes)
api

,classe_consommation_energie,tr001_modele_dpe_type_libelle,annee_construction,_geopoint,latitude,surface_thermique_lot,_i,tr002_type_batiment_description,geo_adresse,_rand,...,classe_estimation_ges,version_methode_dpe,nom_methode_dpe,tv016_departement_code,consommation_energie,date_etablissement_dpe,longitude,_score,_id,geometry
0,F,Location,1974,"50.105335,1.832982",50.105335,100.00,54,Maison Individuelle,12 Place Max Lejeune 80100 Abbeville,786615,...,D,V2012,3CL - DPE,35,430.00,2013-05-06,1.832982,None,Odq6IjxU8z7myJ_DsWJSf,POINT (1.83298 50.10533)
1,D,Vente,1948,"50.095824,1.841555",50.095824,90.00,72,Maison Individuelle,20 Rue de la Plume 80100 Abbeville,27716,...,E,V2012,FACTURE - DPE,80,230.43,2013-05-02,1.841555,None,6th3VqWvwTTZ2doeoKEhc,POINT (1.84156 50.09582)
2,D,Location,1982,"50.109299,1.842122",50.109299,42.80,78,Logement,2 Avenue Aristide Briand 80100 Abbeville,221098,...,B,V2012,3CL - DPE,80,223.39,2013-05-03,1.842122,None,FUt8L1k-W2VQNuouuQbAO,POINT (1.84212 50.10930)
3,N,Vente,1974,"50.109172,1.806312",50.109172,182.05,220,Maison Individuelle,246 Chaussée de Rouvroy 80100 Abbeville,680450,...,N,V2012,3CL - DPE,80,0.00,2013-05-23,1.806312,None,36oVtbKpj76C9vKC-DHcX,POINT (1.80631 50.10917)
4,D,Vente,1974,"50.108204,1.837013",50.108204,146.80,230,Logement,27 Chaussée du Bois 80100 Abbeville,531806,...,E,V2012,3CL - DPE,80,189.78,2013-05-15,1.837013,None,XqBbYI7CzktOxEVb_Ve_T,POINT (1.83701 50.10820)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3616,F,Vente,1981,"45.70891,4.585596",45.708910,135.00,111488,Maison Individuelle,7 Montée des Bruyères 69510 Yzeron,67993,...,G,V2012,3CL - DPE,69,344.65,2016-09-12,4.585596,None,hxx7fyEt3W4zewpeyiYyj,POINT (4.58560 45.70891)
3617,B,Neuf,2016,"45.709107,4.586496",45.709107,122.22,118981,Maison Individuelle,Montée des Bruyères 69510 Yzeron,326477,...,A,V2012,FACTURE - DPE,69,63.52,2016-11-15,4.586496,None,_hwrXzPh80o77KewGC0s6,POINT (4.58650 45.70911)
3619,D,Vente,1948,"45.718229,4.605208",45.718229,115.00,219700,Maison Individuelle,2294 Route de la Brally 69510 Yzeron,148079,...,A,V2012,FACTURE - DPE,69,210.95,2019-01-10,4.605208,None,0pQl2zo6d3NOP1ZIryNc6,POINT (4.60521 45.71823)
3620,B,Neuf,2019,"45.705667,4.591846",45.705667,133.43,259989,Maison Individuelle,Chemin du Planil 69510 Yzeron,808398,...,A,V2012,FACTURE - DPE,69,68.45,2019-10-11,4.591846,None,gquBhuTROFmG2jdRDi_II,POINT (4.59185 45.70567)


In [66]:
api["code_insee_commune_actualise"].nunique()

316

### 1.6 Fusion des tableaux

Il ne nous reste enfin plus qu'à fusionner les tableaux pour obtenir dans une même ligne toutes les données qui nous semblent importantes, à savoir les données de consommations, les données DPE et les données météorologiques annuelles.

In [ ]:
for mot in big_data['geo_adresse'] :
    mot.upper()

data_dpe_2018=filtre_conso_2018.merge(big_data, left_on='Adresse normalisée', right_on='geo_adresse')
data_dpe_2018

In [ ]:
data_reg_2018=data_dpe_2018.merge(meteo_2018, left_on='NOM_M', right_on='Villes')